In [2]:
import piexif
from PIL import Image
import os
import pandas as pd

In [9]:
orgPath='./JPGs'
destPath='./Output'

In [3]:
gpsPos = pd.read_csv('./drone_data2.csv',index_col=0)
gpsPos.head()

,latitude,longitude,altitude
imageName,,,
DJI_0010.JPG,31.9695,35.4877,214.690
DJI_0120.JPG,31.9695,35.4865,214.763
DJI_0170.JPG,31.9695,35.4871,214.753
DJI_0180.JPG,31.9695,35.4872,214.740
DJI_0190.JPG,31.9695,35.4873,214.737


### Definition of working function for lat and lon parsing

In [5]:
def convertLat(lat):
    if lat > 0:
        latMarker = b'N'
    else: 
        latMarker = b'S'
    
    lat = abs(lat)
    degress = int(lat)
    
    tempMinutes = (lat % 1)*60
    minutes = int(tempMinutes)
    
    tempSeconds = (tempMinutes % 1)*60   
    seconds = int(tempSeconds * 10000)
    
    latTuple = ((degress, 1), (minutes, 1), (seconds, 10000))
    return latMarker, latTuple

In [7]:
def convertLon(lon):
    if lon > 0:
        lonMarker = b'E'
    else: 
        lonMarker = b'W'
    
    lon = abs(lon)
    degress = int(lon)
    
    tempMinutes = (lon % 1)*60
    minutes = int(tempMinutes)
    
    tempSeconds = (tempMinutes % 1)*60   
    seconds = int(tempSeconds * 10000)
    
    lonTuple = ((degress, 1), (minutes, 1), (seconds, 10000))
    
    return lonMarker, lonTuple

### Loop over files to save geolocated version

In [10]:
for index, row in gpsPos.iterrows():
    #working with the exif data
    exifDict = {'GPS':
                {0: (2, 4, 0, 0),
                 5: 0}
               }
    latMarker, latTuple = convertLat(row.latitude)
    lonMarker, lonTuple = convertLon(row.longitude)
    exifDict['GPS'][1] = latMarker
    exifDict['GPS'][2] = latTuple
    exifDict['GPS'][3] = lonMarker
    exifDict['GPS'][4] = lonTuple
    exifDict['GPS'][6] = (int(row.altitude*1000),1000)
    #working with the file
    #opening the original file
    orgName=os.path.join(orgPath,index)
    img=Image.open(orgName)
    #write the destination file
    destNname=os.path.join(destPath,index)
    exifBytes = piexif.dump(exifDict)
    img.save(destNname, exif=exifBytes)